In [6]:
from lxml import etree
import requests
import numpy as np
import pandas as pd
import os
import datetime
import threading
import re
from tqdm.notebook import tqdm

In [7]:
def get_table(url):
    #get reponse
    response = requests.get(url)
    #analyze by lxml
    html= etree.HTML(response.text)
    #take first row from table (th)
    title = html.xpath("//table[@bgcolor='#ffffff'][1]//tr//th//text()")
    #take all rows from table (td)
    content = np.array(html.xpath("//table[@bgcolor='#ffffff'][1]//tr//td//text()")).reshape(-1,12)
    #------------------------------------------table整理--------------------------------------------
    def trans(string):
        if type(string) == str:
            string_e = re.sub(r'[▲△]','',string)
            string_e = re.sub(r'[▼▽]','-',string_e)
        else:
            string_e = np.nan
        return string_e
    def time_trans(t):
        if type(t) == str:
            return today+' '+t
        else:
            return np.nan
    table_frame = pd.DataFrame(content,columns=title).drop('股票代號',axis=1)
    table_frame.rename(columns={'選擇':'股票代號'},inplace=True)
    table_frame = table_frame.replace('-',np.nan)
    table_frame['時間'] = table_frame['時間'].apply(time_trans)
    table_frame['漲跌'] = table_frame['漲跌'].apply(trans)
    table_frame['張數'] = table_frame['張數'].str.replace(',','')
    table_frame.iloc[:,3:].astype('float',inplace=True)
    return table_frame

In [8]:
def get_category(number):
    #create date
    global today
    today = str(pd.to_datetime(datetime.datetime.today())).split(' ')[0]
    #url of all category
    response_main = requests.get('https://tw.stock.yahoo.com/h/getclass.php')
    #analyze by lxml
    html_main = etree.HTML(response_main.text)
    #take out text from stock table
    list_c = html_main.xpath("//table[@id='market']//table[@cols='2'][{}]//a/text()".format(number))
    #take out link from stock table
    list_url = html_main.xpath("//table[@id='market']//table[@cols='2'][{}]//a/@href".format(number))
    list_url_f = ['https://tw.stock.yahoo.com'+i for i in list_url]
    catch_list = pd.Series(data=list_url_f,index=list_c)
    #--------------------------save file----------------------------------
    #create folder
    save_place = r'C:\Users\User\Downloads\kaggle\工具包\爬蟲\股市'+today
    if not os.path.exists(save_place):
        os.makedirs(save_place)
    # save file
    for i in tqdm(range(len(catch_list))):
        try:
            address = save_place + '/' + catch_list.index[i] + today + '.xlsx'
            catch = get_table(catch_list[i])
            catch.to_excel(address,index=False)
        except :
            pass

In [9]:
#多線程(繼承threading.Thread)
class table1(threading.Thread):
    def run(self):
        get_category(1)
class table2(threading.Thread):
    def run(self):
        get_category(2)
class table3(threading.Thread):
    def run(self):
        get_category(3)
class table4(threading.Thread):
    def run(self):
        get_category(4)
class table5(threading.Thread):
    def run(self):
        get_category(5)

#主函數
def main():
    #thread
    t1 = table1()
    t2 = table2()
    t3 = table3()
    t4 = table4()
    t5 = table5()
    t1.start()
    t2.start()
    t3.start()
    t4.start()
    t5.start()

In [10]:
main()